In [2]:
import numpy as np
import pandas as pd
from numba import njit, jit

from math import log, sqrt, pi, exp, erf
from scipy.stats import norm

from matplotlib import pyplot as plt
from tqdm import tqdm
from time import perf_counter
import gc

""" API requirements """
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

# Get Data from Yahoo Finance

In [3]:
aapl = yf.Ticker("AAPL")

In [4]:
# get historical market data
hist = aapl.history(period="max")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12,0.100326,0.100762,0.100326,0.100326,469033600,0.0,0.0
1980-12-15,0.095528,0.095528,0.095092,0.095092,175884800,0.0,0.0
1980-12-16,0.088548,0.088548,0.088112,0.088112,105728000,0.0,0.0
1980-12-17,0.090293,0.090729,0.090293,0.090293,86441600,0.0,0.0
1980-12-18,0.092911,0.093347,0.092911,0.092911,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2022-02-28,163.059998,165.419998,162.429993,165.119995,94869100,0.0,0.0
2022-03-01,164.699997,166.600006,161.970001,163.199997,83474400,0.0,0.0
2022-03-02,164.389999,167.360001,162.949997,166.559998,79724800,0.0,0.0


In [63]:
hist.loc[hist.index > np.datetime64('2021-09-01')].Close.pct_change().to_numpy()
hist['prChange'] = hist.Close.pct_change()
hist['log_ret'] = np.log(1 + hist.prChange)

""" Get volatility of last 6 months """
realized_volatility_last_6_months = hist.loc[hist.index > np.datetime64('2021-09-01')].log_ret.std(ddof=1)
print(f'Realized volatility of the last 6 months = {realized_volatility_last_6_months:.9f}')
# df['pct_change'] = df.price.pct_change()
# df['log_ret'] = np.log(df.price) - np.log(df.price.shift(1))

Realized volatility of the last 6 months = 0.016127098


In [46]:
""" Available Dates """
options_list = aapl.options
options_list

('2022-03-11',
 '2022-03-18',
 '2022-03-25',
 '2022-04-01',
 '2022-04-08',
 '2022-04-14',
 '2022-04-22',
 '2022-05-20',
 '2022-06-17',
 '2022-07-15',
 '2022-08-19',
 '2022-09-16',
 '2022-10-21',
 '2022-11-18',
 '2023-01-20',
 '2023-03-17',
 '2023-06-16',
 '2023-09-15',
 '2024-01-19')

In [47]:
opt = aapl.option_chain(options_list[ options_list == "2022-09-16"])
opt

Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice  bid  \
0   AAPL220311C00105000 2022-03-02 20:41:30+00:00   105.0      62.10  0.0   
1   AAPL220311C00110000 2022-03-04 20:28:21+00:00   110.0      53.25  0.0   
2   AAPL220311C00115000 2022-03-04 19:46:47+00:00   115.0      48.10  0.0   
3   AAPL220311C00120000 2022-03-02 20:57:19+00:00   120.0      46.80  0.0   
4   AAPL220311C00125000 2022-02-28 16:26:26+00:00   125.0      40.13  0.0   
5   AAPL220311C00130000 2022-03-04 19:22:35+00:00   130.0      33.38  0.0   
6   AAPL220311C00134000 2022-03-04 16:17:05+00:00   134.0      28.71  0.0   
7   AAPL220311C00135000 2022-03-04 16:17:05+00:00   135.0      27.73  0.0   
8   AAPL220311C00136000 2022-03-04 16:28:01+00:00   136.0      26.65  0.0   
9   AAPL220311C00137000 2022-03-04 15:27:25+00:00   137.0      26.50  0.0   
10  AAPL220311C00138000 2022-03-03 19:10:20+00:00   138.0      29.35  0.0   
11  AAPL220311C00139000 2022-03-01 15:05:42+00:00   139.0     

In [48]:
opt.calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL220311C00105000,2022-03-02 20:41:30+00:00,105.0,62.10,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
1,AAPL220311C00110000,2022-03-04 20:28:21+00:00,110.0,53.25,0.0,0.00,0.0,0.0,70.0,0,0.000010,True,REGULAR,USD
2,AAPL220311C00115000,2022-03-04 19:46:47+00:00,115.0,48.10,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
3,AAPL220311C00120000,2022-03-02 20:57:19+00:00,120.0,46.80,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
4,AAPL220311C00125000,2022-02-28 16:26:26+00:00,125.0,40.13,0.0,0.00,0.0,0.0,2.0,0,0.000010,True,REGULAR,USD
5,AAPL220311C00130000,2022-03-04 19:22:35+00:00,130.0,33.38,0.0,0.00,0.0,0.0,72.0,0,0.000010,True,REGULAR,USD
6,AAPL220311C00134000,2022-03-04 16:17:05+00:00,134.0,28.71,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
7,AAPL220311C00135000,2022-03-04 16:17:05+00:00,135.0,27.73,0.0,0.00,0.0,0.0,21.0,0,0.000010,True,REGULAR,USD
8,AAPL220311C00136000,2022-03-04 16:28:01+00:00,136.0,26.65,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
9,AAPL220311C00137000,2022-03-04 15:27:25+00:00,137.0,26.50,0.0,0.00,0.0,0.0,2.0,0,0.000010,True,REGULAR,USD


In [50]:
""" Closing Price - S0 """
S0 = hist.iloc[-1].Close ; print(f'S0 = {S0}')

""" Options ~6 Months from now """
opt = aapl.option_chain(options_list[ options_list == "2022-09-16"])
opt_puts = opt.puts
opt_puts

S0 = 163.1699981689453


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL220311P00105000,2022-03-04 20:35:57+00:00,105.0,0.01,0.0,0.0,0.0,0.0,68,0,0.500005,False,REGULAR,USD
1,AAPL220311P00110000,2022-03-04 20:52:15+00:00,110.0,0.01,0.0,0.0,0.0,0.0,155,0,0.500005,False,REGULAR,USD
2,AAPL220311P00115000,2022-03-04 20:55:43+00:00,115.0,0.01,0.0,0.0,0.0,0.0,164,0,0.500005,False,REGULAR,USD
3,AAPL220311P00120000,2022-03-04 20:57:57+00:00,120.0,0.03,0.0,0.0,0.0,0.0,197,0,0.500005,False,REGULAR,USD
4,AAPL220311P00125000,2022-03-04 20:58:15+00:00,125.0,0.03,0.0,0.0,0.0,0.0,480,0,0.500005,False,REGULAR,USD
5,AAPL220311P00130000,2022-03-04 20:40:18+00:00,130.0,0.06,0.0,0.0,0.0,0.0,731,0,0.500005,False,REGULAR,USD
6,AAPL220311P00134000,2022-03-04 20:44:02+00:00,134.0,0.07,0.0,0.0,0.0,0.0,215,0,0.500005,False,REGULAR,USD
7,AAPL220311P00135000,2022-03-04 20:59:07+00:00,135.0,0.08,0.0,0.0,0.0,0.0,2377,0,0.250007,False,REGULAR,USD
8,AAPL220311P00136000,2022-03-04 20:53:48+00:00,136.0,0.10,0.0,0.0,0.0,0.0,95,0,0.250007,False,REGULAR,USD
9,AAPL220311P00137000,2022-03-04 20:51:43+00:00,137.0,0.10,0.0,0.0,0.0,0.0,167,0,0.250007,False,REGULAR,USD


In [51]:
""" From that list, puts ~ at the money """
opt_puts.iloc[(opt_puts['strike']-S0).abs().argsort()[:1]]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
27,AAPL220311P00162500,2022-03-04 20:59:59+00:00,162.5,2.71,0.0,0.0,0.0,0.0,32800,0,0.007822,False,REGULAR,USD


In [52]:
""" Treasury Bill """
# t_bill = yf.Tickers(tickers="TMUBMUSD06M") #("AAPL") # TMUBMUSD06M
# t_bill.history()

R = 0.00674
r = np.log(1 + R)

print(f"R    = {R}")
print(f'CC R = {r:.5f}')

R    = 0.00674
CC R = 0.00672


# Trading Days calculation

In [58]:
import pandas_market_calendars as mcal

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Show available calendars
# print(mcal.get_calendar_names())
early = nyse.schedule(start_date='2022-03-07', end_date='2022-09-16')
T = early.shape[0]

print(f"Trading Days until expiration = {T}") ; T = T / 252
print(f"T = {T}")

Trading Days until expiration = 135
T = 0.5357142857142857
